In [40]:
import time

# training dataset
labeled_data_size = 18
unlabeled_data_size = 18
# training hyperparameter
num_train_epochs = 10

# dict_settings
dict_settings = {
    'labeled_data_size':labeled_data_size,
    'unlabeld_data_size':unlabeled_data_size,
    'num_train_epochs':num_train_epochs
}

print(dict_settings)

entire_notebook_t0 = time.time()

{'labeled_data_size': 18, 'unlabeld_data_size': 18, 'num_train_epochs': 10}


In [ ]:
# import tensorflow as tf

# # Get the GPU device name.
# device_name = tf.test.gpu_device_name()

# # The device name should look like the following:
# assert device_name == '/device:GPU:0'
# print('Found GPU at: {}'.format(device_name))

In [ ]:
# import torch

# assert torch.cuda.is_available()

# # Tell torch to use GPU
# device = torch.device("cuda")
# print('Running GPU: {}'.format(torch.cuda.get_device_name()))

In [46]:
# import essential modules
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

import random
import numpy as np
import pandas as pd

import time
import math
import datetime
import seaborn as sns

import os

In [62]:
# Set randon seed
seed = 1

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [31]:
data_path = os.path.join('data')
train_name = 'train_cleaned.csv'
df = pd.read_csv(os.path.join(data_path, train_name), delimiter=',').dropna()

In [32]:
# Mapping the class to label code
# Label Encdoing
class_map={'Demografi':0, 'Ekonomi':1, 'Geografi':2, 'Ideologi':3, 'Pertahanan dan Keamanan':4,
          'Politik':5, 'Sosial Budaya':6, 'Sumber Daya Alam':7}

# Show number of different classes
n_classes = len(class_map)
print('Number of different classes: {}'.format(n_classes))

# Mapping the class names
df['label'] = df['label'].map(class_map)


Number of different classes: 8


In [58]:
df[df['label']==2]

,text,label
238,malam ganjar salawatan istighosah bareng puluh...,2
304,golputpilih capres ganjar pranowo mahfud ganja...,2
529,tiktok bilang anies kepengen bangun jakarta pr...,2
832,kuliah gratis dampaknya tinggal pelosokkalau m...,2
920,ngertipasti panen tanahnya prabowo prabowo bil...,2
1180,gblok pulau reklamasi berjalan dipermasalahina...,2
1607,sumbernya estate gunung kalteng penanggung pra...,2
2046,marah2 anies pulau reklamasi diusik pembuktian...,2
2142,temen komentar anies janji doangmanisaslinya c...,2
2582,ditakedown jakarta bekasivideotron anies muncu...,2


In [33]:
df['label'].value_counts()

label
5    2969
6     584
4     400
3     400
1     366
7     191
0      62
2      18
Name: count, dtype: int64

In [49]:
for i in range(0,8):
    df_temp = df[df['label'] == i].sample(labeled_data_size)
    df_sampled = pd.concat((df_sampled, df_temp))

In [52]:
# Show class countplo
print('Number of data entries: {}'.format(len(df_sampled)))
df_sampled['label'].value_counts()

Number of data entries: 414


label
1    72
0    54
2    54
3    54
4    54
5    54
6    54
7    18
Name: count, dtype: int64

In [63]:
# Randomly choose 15k data entries which will be converted into unlabled data
df_unlabeled = df.sample(unlabeled_data_size)

# Convert class to UNK
df_unlabeled['label'] = 'Geografi'

df_unlabeled

,text,label
2800,hadeh anies poros panislamisme,Geografi
2988,pemahaman mendasar program setara jakarta sala...,Geografi
4134,dukungan ganjarmahfud mengungkap kebenaran sem...,Geografi
4288,musim kampanye mentan berhasil bahan kampanye ...,Geografi
4260,point jakarta mayoritasummat islam memusuhi um...,Geografi
4186,hadiri konsolidasi demokrat sulsel instruksi m...,Geografi
2122,anies concern ketimpangan kepemilikan tanah pr...,Geografi
1743,capres ganjar pranowo memandang rakyat sumber ...,Geografi
1383,masuknya ketua pengajian tasawuf tarekat naqsa...,Geografi
4063,kunci harga rakyat angkat aniesbukan pemilik t...,Geografi


In [65]:
# Split 20% of data from the df_sample as val_data
from sklearn.model_selection import train_test_split

df_labeled, df_val = train_test_split(df_sampled,
                                    test_size=0.2,
                                    random_state=1,
                                    stratify=df_sampled['label'])

for item in df_labeled, df_unlabeled, df_val:
    print(item.shape)

(331, 2)
(18, 2)
(83, 2)


In [66]:
model_name = 'indobenchmark/indobert-base-p1'

from transformers import BertModel, BertTokenizer

# Load BERT tokenizer
transformer = BertModel.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

In [76]:
# Combine df_labeled and df_unlabeled into the train_dataloader
df_train = pd.concat((df_labeled, df_unlabeled))

# Convert UNK to class zero
df_train['label'] = df_train['label'].apply(lambda x: 2 if x=='Geografi' else x)

# Create a new column for label_mask
def generate_label_mask(label):
    if label == 2:
        return 2
    else:
        return 1

df_train['label_mask'] = df_train['label'].apply(generate_label_mask)

# # Show label_mask distribution
# import matplotlib.pyplot as plt
# plt.subplot(1,2,1)
# sns.countplot(df_train['label_mask'])
# plt.title('df_train label mask')

# # Generate label mask for df_val
df_val['label_mask'] = df_val['label'].apply(generate_label_mask)

# # Show label_mask distribution
# plt.subplot(1,2,2)
# sns.countplot(df_val['label_mask'])
# plt.title('df_val label mask')

In [71]:
df_train['label']

1601    3
2145    6
1843    1
2960    0
3562    1
       ..
3969    2
1748    2
1611    2
4746    2
4066    2
Name: label, Length: 349, dtype: int64